## Task 4 - making a dataset generator and patching the images

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
import glob
import os
import cv2
from patchify import patchify
import matplotlib.pyplot as plt
import random

#### Before patching the images, I cropped them using the task 2 code I made and had that as a function I could apply to the images

In [4]:
def crop_petri_dish(image):

    if len(image.shape) > 2:
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray_image = image
    # apply Gaussian Blur to smooth the image
    blurred = cv2.GaussianBlur(gray_image, (9, 9), 0)
    # apply binary threshold
    _, thresh = cv2.threshold(blurred, 10, 255, cv2.THRESH_BINARY) 
    
    # finding contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # filtering out very large contours which might be the image borders
    contours = [cnt for cnt in contours if cv2.contourArea(cnt) < image.shape[0] * image.shape[1] * 0.95]

    # findng the largest contour which will be the petri dish
    largest_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest_contour)
    
    # modify the bounding rectangle to crop more tightly, the margin can be changed
    margin = 90
    x += margin
    y += margin
    w -= 2 * margin
    h -= 2 * margin
    
    # ensuring the modified coordinates are within image bounds
    x = max(x, 0)
    y = max(y, 0)
    w = min(w, image.shape[1] - x)
    h = min(h, image.shape[0] - y)

    # cropping the image based on calculated coordinates
    cropped_image = image[y:y+h, x:x+w]

    return cropped_image, (x, y, w, h)

#### Then I made another function that would apply the same crooping coordinates used for the images but for the masks. Since the cropping function wouldn't work on the masks are there are not contours to find for cropping

In [5]:
def apply_crop(image, bbox):
    x, y, w, h = bbox
    return image[y:y+h, x:x+w]

#### Then I padded the image using the code from self study notebook 7

In [6]:
def padder(image, patch_size):
    """
    Adds padding to an image to make its dimensions divisible by a specified patch size.

    This function calculates the amount of padding needed for both the height and width of an image so that its dimensions become divisible by the given patch size. The padding is applied evenly to both sides of each dimension (top and bottom for height, left and right for width). If the padding amount is odd, one extra pixel is added to the bottom or right side. The padding color is set to black (0, 0, 0).

    Parameters:
    - image (numpy.ndarray): The input image as a NumPy array. Expected shape is (height, width, channels).
    - patch_size (int): The patch size to which the image dimensions should be divisible. It's applied to both height and width.

    Returns:
    - numpy.ndarray: The padded image as a NumPy array with the same number of channels as the input. Its dimensions are adjusted to be divisible by the specified patch size.

    Example:
    - padded_image = padder(cv2.imread('example.jpg'), 128)

    """
    h = image.shape[0]
    w = image.shape[1]
    height_padding = ((h // patch_size) + 1) * patch_size - h
    width_padding = ((w // patch_size) + 1) * patch_size - w

    top_padding = int(height_padding/2)
    bottom_padding = height_padding - top_padding

    left_padding = int(width_padding/2)
    right_padding = width_padding - left_padding

    image = cv2.copyMakeBorder(image, top_padding, bottom_padding, left_padding, right_padding, cv2.BORDER_CONSTANT, value=[0, 0, 0])

    return image

#### Then comes the actual dataset generator, it takes the two directories, those being the original images and masks from the Y23 data set. For the masks it collects file paths that match the specified pattern (ex. "*_root__mask").  

#### Then for each file path it will do the following: load each image and mask, it will resize the images and masks based on the set scaling factor which is 1.0. The apply the crop funciton to crop the images and mask. It will apply the padding function to each mask and image. Then it will use the patchify feature to patch the images and root masks with the set patch size, which at 128. Then it will extract the base name from each image and save them to a directory.

In [ ]:
def save_patches_directly(dataset_type, patch_size, scaling_factor, output_dir, train_ratio=0.8):
    """
    Processes images and root masks, splits them into patches, and saves directly to disk without storing in memory.

    Args:
    - dataset_type (str): Type of dataset to process ('train', 'val').
    - patch_size (int): Size of the patches to extract.
    - scaling_factor (float): Scaling factor for resizing images and masks.
    - output_dir (str): Directory to save the processed patches.
    - train_ratio (float): Ratio of patches to allocate for training (default: 0.8).

    Returns:
    - None
    """
    image_dir = r"C:\Users\emilp\Documents\Buas_Y2\Block_B\data_sets\Y-Combined_dataset\total images"
    mask_dir = r"C:\Users\emilp\Documents\Buas_Y2\Block_B\data_sets\Y-Combined_dataset\total masks"

    # Collect and sort paths for alignment
    image_paths = sorted(glob.glob(os.path.join(image_dir, "*.png")))
    mask_root_paths = sorted(glob.glob(os.path.join(mask_dir, "*.tif")))

    # Make directories
    train_images_dir = os.path.join(output_dir, "train_images", "images")
    train_masks_dir = os.path.join(output_dir, "train_masks", "masks")
    val_images_dir = os.path.join(output_dir, "val_images", "images")
    val_masks_dir = os.path.join(output_dir, "val_masks", "masks")

    os.makedirs(train_images_dir, exist_ok=True)
    os.makedirs(train_masks_dir, exist_ok=True)
    os.makedirs(val_images_dir, exist_ok=True)
    os.makedirs(val_masks_dir, exist_ok=True)

    for img_path, root_path in zip(image_paths, mask_root_paths):
        # Load and process images and root masks
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        root_mask = cv2.imread(root_path, cv2.IMREAD_GRAYSCALE)

        # Resize
        image = cv2.resize(image, (0, 0), fx=scaling_factor, fy=scaling_factor)
        root_mask = cv2.resize(root_mask, (0, 0), fx=scaling_factor, fy=scaling_factor)

        image, bbox = crop_petri_dish(image)
        root_mask = apply_crop(root_mask, bbox)

        image = padder(image, patch_size)
        root_mask = padder(root_mask, patch_size)

        # Patchify
        image_patches = patchify(image, (patch_size, patch_size), step=patch_size).reshape(-1, patch_size, patch_size)
        root_patches = patchify(root_mask, (patch_size, patch_size), step=patch_size).reshape(-1, patch_size, patch_size)

        # Split into train and validation
        indices = np.arange(len(image_patches))
        train_indices, val_indices = train_test_split(indices, train_size=train_ratio, random_state=42)

        for split, indices, img_dir, mask_dir in [
            ("train", train_indices, train_images_dir, train_masks_dir),
            ("val", val_indices, val_images_dir, val_masks_dir),
        ]:
            for idx in indices:
                base_name = os.path.splitext(os.path.basename(img_path))[0]
                img_patch_path = os.path.join(img_dir, f"{base_name}_patch_{idx}.png")
                mask_patch_path = os.path.join(mask_dir, f"{base_name}_patch_{idx}.png")

                    # Write both image and mask patch tx`o disk
                cv2.imwrite(img_patch_path, image_patches[idx].astype(np.uint8))
                cv2.imwrite(mask_patch_path, root_patches[idx].astype(np.uint8) * 255)  # Enhance visibility for masks

# Set parameters
output_directory = r"C:\Users\emilp\Documents\Buas_Y2\Block_B\data_sets\Y-Combined_dataset\placeholder_256"
dataset_type = "train"
patch_size = 256
scaling_factor = 1.0

# Save patches directly
save_patches_directly(dataset_type, patch_size, scaling_factor, output_directory)

#### After which the list of original image names that was saved is used for the created patches. Every image generates around 500 patches, each patch having the same name as the original image with a number indicator at the end. It is also split into training and validation sets with a ratio of 80/20. And then saved into the requested folder structure. 